
# 漢字・アルファベットからカタカナを推測するモデルを学習する
オープンソース辞書及び，Wikipediaから作成した漢字とカタカナのペアデータを用いて，Transformer+Seq2Seqのモデルで，スクラッチで学習する。

入力の文字（漢字・アルファベットなど）は一文字ずつに分割，また，出力の文字（カタカナ）も一文字ずつに分割して，学習する。



## データ準備
データは，[dict](../dict)で作成した姓名辞書データのoss.json及びseimei.jsonと，Wikipediaから抜き出した，漢字姓名とカタカナ姓名（スペース区切りのもの）を用いて，学習に使用した。単語単位である姓名辞書データと，Wikipediaのスペースで区切られた姓名を用いることで，姓名で漢字とカナが入力された場合の学習を可能とした。

また，ひらがなとカタカナ，


In [16]:
!date

2025年  6月  6日 金曜日 19:07:30 JST


In [17]:
# スクリプトで使う環境変数をセット
import os
os.environ['VER']='1.7o'
os.environ['DIR']='dataset/dataset.'+os.environ['VER']
os.environ['MODEL']='model/model.'+os.environ['VER']
os.environ["LOG"]='logs/logs.'+os.environ['VER']
os.environ['RATIO']="0.01"
os.environ['WIKI']="../dict/oss/wikipedia/wikiname.txt" # Wikipediaの概要欄から取得した姓名の漢字・アルファベットとカタカナのペアデータ。このファイルの姓名をスペースで区切って丹後他院位にしたものは，辞書ファイルに追加済み
os.environ['DICT']='../dict/data/data.'+os.environ['VER']
os.makedirs(os.environ['DIR'],exist_ok=True)
os.makedirs(os.environ['MODEL'],exist_ok=True)

## 漢字・アルファベットとカタカナのペアから学習する
学習は，訓練用データを用いて学習し，１エポックごとに開発用データ(valid.json)でLossを確認する。


### パラメタ
- emb_size 
入力，出力の文字のエンベッドのDimension

- nhead    
マルチヘッド数

- ffn_hid_dim    
FFNのDimension

- num_encoder_layer    
エンコードレイヤーの数

- num_decoder_layer    
デコードレイヤーの数

- lr     
学習率

- dropout    
ドロップアウトの割合, 0-1

- num_epochs    
何周学習データを用いて学習を行うか

- device    
mps,cpu,cudaから選択
  - mps        
   アップルシリコンを搭載したマシンで実行する際に選択
  - cuda         
   CUDAが利用できる環境で選択
  - cpu        
   上記以外は，CPUモードを選択

- earlystop_patient    
開発用データでlossが下がらなくなってearlystop_patient回計算が終了した場合に，num_epochs数以下でも，計算を終了させる

- output_dir    
モデルの出力ディレクトリ

- tensorboard_logdir    
tensorboard形式のログの出力ディレクトリ。学習状況を確認するためには`tensorboard --logdir logs`を実行後，ブラウザでhttp://localhost:6000/から確認
 



- prefix    
jsonl形式のデータのprefix

- source_lang    
jsonl形式のデータのsourceのキー

- target_lang    
jsonl形式のデータのtargetのキー

- train_file    
訓練用データのJSONLファイル

- valid_file    
開発用データのJSONLファイル

In [18]:
# 訓練実行

!python ./char_model.py \
  --emb_size 512 \
  --nhead 8 \
  --ffn_hid_dim 2048 \
  --batch_size 128 \
  --num_encoder_layers 8 \
  --num_decoder_layers 8 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 152 \
  --device cuda \
  --earlystop_patient 3 \
  --output_dir $MODEL \
  --tensorboard_logdir $LOG \
  --prefix translation \
  --source_lang kanji \
  --target_lang kana \
  --train_file $DIR/train.jsonl \
  --valid_file $DIR/valid.jsonl


num_epochs:152
Epoch: 1, Train loss: 3.285, Val loss: 2.586, Epoch time = 6054.556s
Epoch: 2, Train loss: 2.481, Val loss: 2.002, Epoch time = 6055.001s
Epoch: 3, Train loss: 2.100, Val loss: 1.672, Epoch time = 6046.985s
Epoch: 4, Train loss: 1.846, Val loss: 1.434, Epoch time = 6040.936s
Epoch: 5, Train loss: 1.643, Val loss: 1.239, Epoch time = 6080.532s
Epoch: 6, Train loss: 1.477, Val loss: 1.098, Epoch time = 6031.828s
Epoch: 7, Train loss: 1.349, Val loss: 0.990, Epoch time = 6027.014s
Epoch: 8, Train loss: 1.247, Val loss: 0.906, Epoch time = 6038.015s
Epoch: 9, Train loss: 1.163, Val loss: 0.835, Epoch time = 6025.858s
Epoch: 10, Train loss: 1.092, Val loss: 0.777, Epoch time = 6021.853s
Epoch: 11, Train loss: 1.031, Val loss: 0.734, Epoch time = 6025.843s
Epoch: 12, Train loss: 0.977, Val loss: 0.693, Epoch time = 6021.040s
Epoch: 13, Train loss: 0.930, Val loss: 0.661, Epoch time = 6045.032s
Epoch: 14, Train loss: 0.887, Val loss: 0.627, Epoch time = 6022.027s
Epoch: 15, Tra

In [24]:
%load_ext tensorboard
%tensorboard --logdir logs/logs.$VER

## モデルを検証する
作成したモデルを，検証用データを用いて検証し，正解率を算出する


## パラメタ

- test_file
jsonl形式の検証用データファイル

- model_file
transformer_model.pyで作成したモデルファイル。

- outfile
検証結果を格納するファイル

- device
cpu限定

- nbest
検証用データの漢字姓名を入力データとした時に，モデルが出力するカタカナ姓名を確率の高い方からnbest個出力する。beam_width以下。searchパラメタがbothかbeamの時有効

- beam_width
ビームサーチのビーム幅。searchパラメタがbothかbeamの時有効

- max_len
出力するカタカナ姓名の最大長さ

- search
greedy,beam,bothから選択
-- greedy
貪欲法による探索
-- beam
ビームサーチによる探索
-- both
貪欲砲，ビームサーチ両方の探索



In [25]:
# 検証用データを実行


!python generate_batch.py \
  --test_file $DIR/test.jsonl \
  --model_file $MODEL/checkpoint_best.pt \
  --outfile $DIR/generate.txt \
  --device cpu \
  --nbest 5 \
  --beam_width 5 \
  --max_len 100 \
  --search greedy





In [26]:
# 検証結果を評価
import pandas as pd
import os
dr=os.environ["DIR"]
df = pd.read_csv(f'{dr}/generate.txt',encoding='utf-8',sep="\t")
tgt = df['tgt'].tolist()
pred = df['pred'].tolist()
ok=0
for t,p in zip(tgt,pred):
    if t == p:
        ok+=1

print(f'acc={ok/len(tgt)}')


acc=0.6069319871997115


# モデルをJavaのDJL用に変換する
Pytorchで学習したモデルをJavaで使えるようにモデルの変換を行う。



In [27]:
# 漢字からカナ
!python convert_jitscript.py  \
    --model_file=$MODEL/checkpoint_best.pt \
    --model_script=$MODEL/script_$VER.pt \
    --encoder=$MODEL/encoder_$VER.pt \
    --decoder=$MODEL/decoder_$VER.pt \
    --positional_encoding=$MODEL/positional_encoding_$VER.pt \
    --generator=$MODEL/generator_$VER.pt \
    --src_tok_emb=$MODEL/src_tok_emb_$VER.pt \
    --tgt_tok_emb=$MODEL/tgt_tok_emb_$VER.pt \
    --vocab_src=$MODEL/vocab_src_$VER.txt \
    --vocab_tgt=$MODEL/vocab_tgt_$VER.txt \
    --params=$MODEL/params_$VER.json \
    --device=cpu



In [28]:
!date

2025年  6月 10日 火曜日 11:41:45 JST
